In [27]:
import sqlite3
import pandas as pd

# Load the CSV file
data = pd.read_csv('/scratch/students/saydalie/venice_llm/data/data_20240221/clean/catastici.csv')

# Create a column for the unique owner name
data['Owner_ID'] = data['Owner_First_Name'] + ' ' + data['Owner_Family_Name']

# Make the owner names unique by appending a counter for duplicates
data['Owner_ID'] = data.groupby('Owner_ID').cumcount().astype(str).replace('0', '') + data['Owner_ID']

# Prepare the values for the database
values_sql = [(row['Owner_ID'], row['Owner_First_Name'], row['Owner_Family_Name'], row['Property_Type'], row['Rent_Income'], row['Property_Location']) for _, row in data.iterrows()]

# Connect to SQLite (or create the database file if it doesn't exist)
conn = sqlite3.connect('/scratch/students/saydalie/venice_llm/data/data_20240221/clean/catastici.db')
cursor = conn.cursor()

# Create a table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS catastici (
        ID INTEGER PRIMARY KEY AUTOINCREMENT,
        Owner_ID INTEGER,
        Owner_First_Name TEXT,
        Owner_Family_Name TEXT,
        Property_Type TEXT,
        Rent_Income INTEGER,
        Property_Location TEXT
    )
''')

# Insert data into the table
cursor.executemany('''
    INSERT INTO catastici (Owner_ID, Owner_First_Name, Owner_Family_Name, Property_Type, Rent_Income, Property_Location)
    VALUES (?, ?, ?, ?, ?, ?)
''', values_sql)

# Commit and close the connection
conn.commit()
conn.close()

print("Database created and data inserted successfully.")

Database created and data inserted successfully.


In [28]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:////scratch/students/saydalie/venice_llm/data/data_20240221/clean/catastici.db")
# db = SQLDatabase.from_uri("sqlite:////scratch/students/saydalie/venice_llm/text_to_sql/experiments/experiment_6/catastici.db")

# test DB
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM catastici LIMIT 1;")

sqlite
['catastici', 'sqlite_sequence']


"[(1, 'liberal campi', 'liberal', 'campi', 'casa e bottega da barbier', 70, 'campo vicino alla chiesa')]"

In [29]:
from collections import Counter
import pandas as pd
import ast
import re
import warnings
warnings.filterwarnings('ignore')

def clean_query(sql_query):
    """clean the output"""
    # change to list
    sql_query_list = ast.literal_eval(sql_query)
    
    # split on ;
    sql_query_list = [query.split(';')[0].strip() + ';' for query in sql_query_list]
    # sql_query_list = [query.split(';')[0].split('[/SQL]')[0].strip() + ';' for query in sql_query_list]
    
    # replace ' with ''
    sql_query_list = [re.sub(r"([a-z])'([a-z])",r"\1''\2", query) for query in sql_query_list]

    return sql_query_list

def check_sql_executability(query, db):
    try:
        return db.run(query)
    except Exception as e:
        return str(e)
     
def find_most_common_answer(answers):
    answers = ["ERROR" if "error" in answer else answer for answer in answers]
    most_common_answer, most_common_count = Counter(answers).most_common(1)[0]
    if most_common_answer == "ERROR":
        if most_common_count == 4:
            return most_common_answer
        else:
            return Counter(answers).most_common(2)[1][0]

    return most_common_answer

def clean_answer(answer, to_replace = ['[', ']', '(', ',)', "'", ')']):
    pattern = '|'.join(map(re.escape, to_replace))
    cleaned_answer = re.sub(pattern, '', answer)
    return [ans.strip() for ans in cleaned_answer.split(',')]

def ngram_overlap(true_answer, generated_answer):
    """Calculates 1-gram overlap"""
    # Convert lists to sets to remove duplicates
    true_set = set(true_answer)
    generated_set = set(generated_answer)
    
    # Calculate the intersection of the sets
    overlap_count = len(true_set.intersection(generated_set))
    
    # Calculate the percentage of overlap
    overlap_percentage = overlap_count / len(true_set) if len(true_set) > 0 else 0.0
    
    return round(overlap_percentage, 3)

# Zero shot

In [30]:
# import the dataset
query_res = pd.read_csv('./test_data_generated_0.csv')

# clean output
for idx, row in query_res.iterrows():
    query_list_clean = clean_query(row['generated_query'])
    final_out = None
    answers = []
    for out in query_list_clean:
        answers.append(check_sql_executability(out, db))
    answer = find_most_common_answer(answers)
    
    if answer != "ERROR":
        final_out = query_list_clean[answers.index(answer)]
    if final_out == None:
        final_out = query_list_clean[0]
        answer = check_sql_executability(final_out, db)
    query_res.loc[idx,'generated_answer'] = answer
    query_res.loc[idx,'generated_query'] = final_out

    true_answer_new = check_sql_executability(row['true_query'], db)
    query_res.loc[idx,'true_answer'] = true_answer_new
    
query_res.loc[(query_res['generated_answer'].str.contains("error")), 'output'] = 'ERROR'
query_res.loc[(query_res['generated_answer']==query_res['true_answer']), 'output'] = 'EM'
print(query_res['output'].value_counts())

output
EM       198
ERROR      7
Name: count, dtype: int64


In [31]:
query_wrong = query_res[query_res.output.isna()]
query_wrong.true_answer.fillna("",inplace=True)

query_wrong['generated_answer_clean'] = query_wrong['generated_answer'].apply(clean_answer)
query_wrong['true_answer_clean'] = query_wrong['true_answer'].apply(clean_answer)

n_gram = []
for _, row in query_wrong.iterrows():
    n_gram.append(ngram_overlap(row['true_answer_clean'], row['generated_answer_clean']))
query_wrong['n_gram_overlap'] = n_gram
query_wrong['output'] = query_wrong['n_gram_overlap']>0.33

query_wrong['output'].value_counts()

output
True     160
False    135
Name: count, dtype: int64

In [32]:
columns = ['question_id','level_len','level_nest','question','true_query','true_answer','matched_contents','generated_query','generated_answer','output']
query_0 = pd.concat([query_res[query_res['output'].notna()], query_wrong], axis=0)[columns].rename({
        'generated_query':'generated_query_0', 
        'generated_answer':'generated_answer_0', 
        'output':'output_0'},axis=1
    )
query_0['output_0'].value_counts()

output_0
EM       198
True     160
False    135
ERROR      7
Name: count, dtype: int64

# 3-shot

In [7]:
# import the dataset
query_res = pd.read_csv('./test_data_generated_3.csv')

# clean output
for idx, row in query_res.iterrows():
    query_list_clean = clean_query(row['generated_query'])
    final_out = None
    answers = []
    for out in query_list_clean:
        answers.append(check_sql_executability(out, db))
    answer = find_most_common_answer(answers)
    
    if answer != "ERROR":
        final_out = query_list_clean[answers.index(answer)]
    if final_out == None:
        final_out = query_list_clean[0]
        answer = check_sql_executability(final_out, db)
    query_res.loc[idx,'generated_answer'] = answer
    query_res.loc[idx,'generated_query'] = final_out

    true_answer_new = check_sql_executability(row['true_query'], db)
    query_res.loc[idx,'true_answer'] = true_answer_new
    
query_res.loc[(query_res['generated_answer'].str.contains("error")), 'output'] = 'ERROR'
query_res.loc[(query_res['generated_answer']==query_res['true_answer']), 'output'] = 'EM'
query_res['output'].value_counts()

output
EM    285
Name: count, dtype: int64

In [8]:
query_wrong = query_res[query_res.output.isna()]
query_wrong.true_answer.fillna("",inplace=True)

query_wrong['generated_answer_clean'] = query_wrong['generated_answer'].apply(clean_answer)
query_wrong['true_answer_clean'] = query_wrong['true_answer'].apply(clean_answer)

n_gram = []
for _, row in query_wrong.iterrows():
    n_gram.append(ngram_overlap(row['true_answer_clean'], row['generated_answer_clean']))
query_wrong['n_gram_overlap'] = n_gram
query_wrong['output'] = query_wrong['n_gram_overlap']>0.33

query_wrong['output'].value_counts()

output
False    115
True     100
Name: count, dtype: int64

In [9]:
columns = ['question_id','level_len','level_nest','question','true_query','true_answer','matched_contents','generated_query','generated_answer','output']
query_3 = pd.concat([query_res[query_res['output'].notna()], query_wrong], axis=0)[columns].rename({
        'generated_query':'generated_query_3', 
        'generated_answer':'generated_answer_3', 
        'output':'output_3'},axis=1
    )
query_3['output_3'].value_counts()

output_3
EM       285
False    115
True     100
Name: count, dtype: int64

# 5-shot

In [10]:
# import the dataset
query_res = pd.read_csv('./test_data_generated_5.csv')

# clean output
for idx, row in query_res.iterrows():
    query_list_clean = clean_query(row['generated_query'])
    final_out = None
    answers = []
    for out in query_list_clean:
        answers.append(check_sql_executability(out, db))
    answer = find_most_common_answer(answers)
    
    if answer != "ERROR":
        final_out = query_list_clean[answers.index(answer)]
    if final_out == None:
        final_out = query_list_clean[0]
        answer = check_sql_executability(final_out, db)
    query_res.loc[idx,'generated_answer'] = answer
    query_res.loc[idx,'generated_query'] = final_out

    true_answer_new = check_sql_executability(row['true_query'], db)
    query_res.loc[idx,'true_answer'] = true_answer_new

query_res.loc[(query_res['generated_answer'].str.contains("error")), 'output'] = 'ERROR'
query_res.loc[(query_res['generated_answer']==query_res['true_answer']), 'output'] = 'EM'
query_res['output'].value_counts()

output
EM    289
Name: count, dtype: int64

In [11]:
query_wrong = query_res[query_res.output.isna()]
query_wrong.true_answer.fillna("",inplace=True)

query_wrong['generated_answer_clean'] = query_wrong['generated_answer'].apply(clean_answer)
query_wrong['true_answer_clean'] = query_wrong['true_answer'].apply(clean_answer)

n_gram = []
for _, row in query_wrong.iterrows():
    n_gram.append(ngram_overlap(row['true_answer_clean'], row['generated_answer_clean']))
query_wrong['n_gram_overlap'] = n_gram
query_wrong['output'] = query_wrong['n_gram_overlap']>0.33

query_wrong['output'].value_counts()

output
True     108
False    103
Name: count, dtype: int64

In [12]:
columns = ['question_id','level_len','level_nest','question','true_query','true_answer','matched_contents','generated_query','generated_answer','output']
query_5 = pd.concat([query_res[query_res['output'].notna()], query_wrong], axis=0)[columns].rename({
        'generated_query':'generated_query_5', 
        'generated_answer':'generated_answer_5', 
        'output':'output_5'},axis=1
    )
query_5['output_5'].value_counts()

output_5
EM       289
True     108
False    103
Name: count, dtype: int64

# 7-shot

In [13]:
# import the dataset
query_res = pd.read_csv('./test_data_generated_7.csv')

# clean output
for idx, row in query_res.iterrows():
    query_list_clean = clean_query(row['generated_query'])
    final_out = None
    answers = []
    for out in query_list_clean:
        answers.append(check_sql_executability(out, db))
    answer = find_most_common_answer(answers)
    
    if answer != "ERROR":
        final_out = query_list_clean[answers.index(answer)]
    if final_out == None:
        final_out = query_list_clean[0]
        answer = check_sql_executability(final_out, db)
    query_res.loc[idx,'generated_answer'] = answer
    query_res.loc[idx,'generated_query'] = final_out

    true_answer_new = check_sql_executability(row['true_query'], db)
    query_res.loc[idx,'true_answer'] = true_answer_new

query_res.loc[(query_res['generated_answer'].str.contains("error")), 'output'] = 'ERROR'
query_res.loc[(query_res['generated_answer']==query_res['true_answer']), 'output'] = 'EM'
query_res['output'].value_counts()

output
EM    292
Name: count, dtype: int64

In [14]:
query_wrong = query_res[query_res.output.isna()]
query_wrong.true_answer.fillna("",inplace=True)

query_wrong['generated_answer_clean'] = query_wrong['generated_answer'].apply(clean_answer)
query_wrong['true_answer_clean'] = query_wrong['true_answer'].apply(clean_answer)

n_gram = []
for _, row in query_wrong.iterrows():
    n_gram.append(ngram_overlap(row['true_answer_clean'], row['generated_answer_clean']))
query_wrong['n_gram_overlap'] = n_gram
query_wrong['output'] = query_wrong['n_gram_overlap']>0.33

query_wrong['output'].value_counts()

output
True     105
False    103
Name: count, dtype: int64

In [15]:
columns = ['question_id','level_len','level_nest','question','true_query','true_answer','matched_contents','generated_query','generated_answer','output']
query_7 = pd.concat([query_res[query_res['output'].notna()], query_wrong], axis=0)[columns].rename({
        'generated_query':'generated_query_7', 
        'generated_answer':'generated_answer_7', 
        'output':'output_7'},axis=1
    )
query_7['output_7'].value_counts()

output_7
EM       292
True     105
False    103
Name: count, dtype: int64

# 5-shot - 15b model

In [14]:
# import the dataset
query_res = pd.read_csv('./test_data_generated_5_15b.csv')

# clean output
for idx, row in query_res.iterrows():
    query_list_clean = clean_query(row['generated_query'])
    final_out = None
    answers = []
    for out in query_list_clean:
        answers.append(check_sql_executability(out, db))
    answer = find_most_common_answer(answers)
    
    if answer != "ERROR":
        final_out = query_list_clean[answers.index(answer)]
    if final_out == None:
        final_out = query_list_clean[0]
        answer = check_sql_executability(final_out, db)
    query_res.loc[idx,'generated_answer'] = answer
    query_res.loc[idx,'generated_query'] = final_out
    
query_res.loc[(query_res['generated_answer'].str.contains("error")), 'output'] = 'ERROR'
query_res.loc[(query_res['generated_answer']==query_res['true_answer']), 'output'] = 'EM'
query_res['output'].value_counts()

output
EM    306
Name: count, dtype: int64

In [15]:
query_wrong = query_res[query_res.output.isna()]
query_wrong.true_answer.fillna("",inplace=True)

query_wrong['generated_answer_clean'] = query_wrong['generated_answer'].apply(clean_answer)
query_wrong['true_answer_clean'] = query_wrong['true_answer'].apply(clean_answer)

n_gram = []
for _, row in query_wrong.iterrows():
    n_gram.append(ngram_overlap(row['true_answer_clean'], row['generated_answer_clean']))
query_wrong['n_gram_overlap'] = n_gram
query_wrong['output'] = query_wrong['n_gram_overlap']>0.33

query_wrong['output'].value_counts()

output
True     98
False    96
Name: count, dtype: int64

In [16]:
columns = ['question_id','level_len','level_nest','question','true_query','true_answer','matched_contents','generated_query','generated_answer','output']
query_res = pd.concat([query_res[query_res['output'].notna()], query_wrong], axis=0)[columns]
query_res['output'].value_counts()

output
EM       306
True      98
False     96
Name: count, dtype: int64

In [17]:
# difficulty
query_res['output_binary'] = (query_res.output!=False)

print(query_res.groupby('level_len')['output_binary'].mean())
print(query_res.groupby('level_nest')['output_binary'].mean())
query_res.drop('output_binary',axis=1,inplace=True)

level_len
0    0.90
1    0.44
Name: output_binary, dtype: float64
level_nest
0    0.898734
1    0.466667
Name: output_binary, dtype: float64


# All

In [48]:
query

,question_id,level_len,level_nest,question,true_query,true_answer,matched_contents,generated_query_0,generated_answer_0,output_0,generated_query_3,generated_answer_3,output_3,generated_query_5,generated_answer_5,output_5,generated_query_7,generated_answer_7,output_7
0,0,0,0,Specify all the property types that are encaps...,"SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('...",NaN,SELECT DISTINCT catastici.Property_Type\nFROM ...,"[('casa e bottega da barbier',), ('casa',), ('...",EM,"SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('...",EM,"SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('...",EM,"SELECT DISTINCT ""Property_Type""\nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('...",EM
1,1,0,0,How many properties are recorded in the dataset?,SELECT COUNT(*) AS Total_Properties\nFROM cata...,"[(15674,)]",NaN,SELECT COUNT(catastici.ID)\nFROM catastici;,"[(15674,)]",EM,SELECT COUNT(*) FROM catastici;,"[(15674,)]",EM,SELECT COUNT(*) \nFROM catastici;,"[(15674,)]",EM,SELECT COUNT(*) \nFROM catastici;,"[(15674,)]",EM
2,1,0,0,"Regarding the dataset, what is the count of li...",SELECT COUNT(*) AS Total_Properties\nFROM cata...,"[(15674,)]",NaN,SELECT COUNT(catastici.ID)\nFROM catastici;,"[(15674,)]",EM,SELECT COUNT(*) \nFROM catastici;,"[(15674,)]",EM,SELECT COUNT(*) FROM catastici;,"[(15674,)]",EM,SELECT COUNT(*) \nFROM catastici;,"[(15674,)]",EM
3,1,0,0,What is the total number of properties listed ...,SELECT COUNT(*) AS Total_Properties\nFROM cata...,"[(15674,)]",NaN,SELECT COUNT(*)\nFROM catastici;,"[(15674,)]",EM,SELECT COUNT(*) AS num_properties FROM catastici;,"[(15674,)]",EM,SELECT COUNT(*) \nFROM catastici;,"[(15674,)]",EM,SELECT COUNT(*) \nFROM catastici;,"[(15674,)]",EM
4,3,0,0,How many properties have a rental income lower...,SELECT COUNT(*) \nFROM catastici \nWHERE Rent_...,"[(9768,)]",NaN,SELECT COUNT(*)\nFROM catastici\nWHERE Rent_In...,"[(9768,)]",EM,SELECT COUNT(*) \nFROM catastici \nWHERE Rent_...,"[(9768,)]",EM,SELECT COUNT(*) \nFROM catastici \nWHERE Rent_...,"[(9768,)]",EM,SELECT COUNT(*) \nFROM catastici \nWHERE Rent_...,"[(9768,)]",EM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,99,1,1,How does the rent income fluctuate on average ...,SELECT AVG(Variance) AS Avg_Rent_Income_Varian...,"[(6150.230341110285,)]",NaN,"SELECT catastici.Property_Location, AVG(catast...","[('calle della crea', 2996.0), (""calle in camp...",False,"SELECT ""Property_Location"", AVG(""Rent_Income"")...",[('[appartamento] li formenti sopra la strada ...,False,"SELECT ""Property_Location"", AVG(""Rent_Income"")...",[('[appartamento] li formenti sopra la strada ...,False,"SELECT ""Property_Location"", AVG(""Rent_Income"")...",[('[appartamento] li formenti sopra la strada ...,False
496,99,1,1,"Across various locations, what variation exist...",SELECT AVG(Variance) AS Avg_Rent_Income_Varian...,"[(6150.230341110285,)]",NaN,"SELECT catastici.Property_Location, AVG(catast...","[('calle della crea', 2996.0), (""calle in camp...",False,"SELECT ""Property_Location"", AVG((""Rent_Income""...",[('[appartamento] li formenti sopra la strada ...,False,"SELECT ""Property_Location"", AVG(""Rent_Income"")...",[('[appartamento] li formenti sopra la strada ...,False,"SELECT ""Property_Location"", AVG(""Rent_Income"")...",[('[appartamento] li formenti sopra la strada ...,False
497,99,1,1,What's the range of differences in average ren...,SELECT AVG(Variance) AS Avg_Rent_Income_Varian...,"[(6150.230341110285,)]",NaN,"SELECT Property_Location, AVG(Rent_Income) AS ...","[('calle della crea', 2996.0), (""calle in camp...",False,"SELECT ""Property_Location"", MAX(""Rent_Income"")...",[('[appartamento] li formenti sopra la strada ...,False,"SELECT ""Property_Location"", MAX(""Rent_Income"")...",[('[appartamento] li formenti sopra la strada ...,False,"SELECT ""Pr

In [40]:
query['output_7'].isin(['EM', True]).mean()

0.794

In [33]:
query = pd.merge(query_0, query_3, on=['question_id','level_len','level_nest','question','true_query','true_answer','matched_contents'])
query = pd.merge(query, query_5, on=['question_id','level_len','level_nest','question','true_query','true_answer','matched_contents'])
query = pd.merge(query, query_7, on=['question_id','level_len','level_nest','question','true_query','true_answer','matched_contents'])

In [35]:
print(query['output_0'].value_counts())
print(query['output_3'].value_counts())
print(query['output_5'].value_counts())
print(query['output_7'].value_counts())

output_0
EM       198
True     160
False    135
ERROR      7
Name: count, dtype: int64
output_3
EM       285
False    115
True     100
Name: count, dtype: int64
output_5
EM       289
True     108
False    103
Name: count, dtype: int64
output_7
EM       292
True     105
False    103
Name: count, dtype: int64


In [18]:
# difficulty
query['output_binary'] = (query.output_5!=False)

print(query.groupby('level_len')['output_binary'].mean())
print(query.groupby('level_nest')['output_binary'].mean())
query.drop('output_binary',axis=1,inplace=True)

level_len
0    0.8775
1    0.4600
Name: output_binary, dtype: float64
level_nest
0    0.883544
1    0.457143
Name: output_binary, dtype: float64


In [23]:
query.to_csv('test_data_eval_5.csv', index=False)

In [21]:
for idx, row in query_wrong[query_wrong.output == False].sample(5).iterrows():
    print(f"{idx} - {row['question_id']}")
    print(f"Question: {row['question']}")
    # print(f"N-gram: {row['n_gram_overlap']}")
    print(f"True Answer: {row['true_answer']}")
    print(f"Generated Answer: {row['generated_answer']}")
    print()
    print('True SQL:')
    print(row['true_query'])
    print('Generated SQL:')
    print(row['generated_query'])
    print('\n\n') 

428 - 85
Question: What sum of rent income is produced by the elite five earners?
True Answer: [(23954,)]
Generated Answer: [(17038,)]

True SQL:
SELECT SUM("Rent_Income") AS Total_Rent_Income
FROM catastici
WHERE ("Owner_First_Name", "Owner_Family_Name") IN (
SELECT "Owner_First_Name", "Owner_Family_Name"
FROM catastici
ORDER BY "Rent_Income" DESC
LIMIT 5
);
Generated SQL:
SELECT SUM("Rent_Income") AS Total_Rent_Income
FROM catastici
WHERE "Owner_ID" IN (
	SELECT "Owner_ID"
	FROM catastici
	GROUP BY "Owner_ID"
	ORDER BY COUNT("Owner_ID") DESC
	LIMIT 5
);



456 - 91
Question: What is the number of property holders who earn over 100 ducati from rents in total?
True Answer: [(1113,)]
Generated Answer: [(489,)]

True SQL:
SELECT COUNT(*) AS owners_with_more_than_100_income
FROM (
SELECT DISTINCT "Owner_First_Name", "Owner_Family_Name"
FROM catastici
GROUP BY "Owner_First_Name", "Owner_Family_Name"
HAVING SUM("Rent_Income") > 100
) AS owners_with_income_above_100;
Generated SQL:
SELECT CO